In [ ]:
"""
Main Entry Point
"""
from pathlib import Path

import pandas as pd # type: ignore

import src.fetch_data
import src.core_transform
import cdutils.pkey_sqlite # type: ignore
import cdutils.hhnbr # type: ignore
from src._version import __version__

# def main(production_flag: bool=False):
#     if production_flag:
#         BASE_PATH = Path(r'\\00-DA1\Home\Share\Line of Business_Shared Services')
#         assert "prod" in __version__, (f"Cannot run in production mode without 'prod' in the __version__")
#     else:
#         BASE_PATH = Path('.')
    
data = src.fetch_data.fetch_data()

# Core transformation pipeline
raw_data = src.core_transform.main_pipeline(data)

# Raw data with pkey appended
raw_data = cdutils.pkey_sqlite.add_pkey(raw_data)

# Attach HH number
househldacct = data['househldacct'].copy()
raw_data = cdutils.hhnbr.add_hh_nbr(raw_data, househldacct)

# Attach dimensions & segments
DIM_PATH = Path('./assets/Dimensions and Segments.xlsx')
dim_df = pd.read_excel(io=DIM_PATH, sheet_name='Account Dimensions')
df = src.core_transform.add_dimensions(main_df=raw_data, dim_df=dim_df)

# # # Output to excel (raw data)
# OUTPUT_PATH = BASE_PATH / Path('./output/standard_report.xlsx')
# raw_data.to_excel(OUTPUT_PATH, sheet_name='Sheet1', index=False)

# # # Format excel
# # src.output_to_excel.format_excel_file(OUTPUT_PATH)

# if __name__ == '__main__':
# print(f"Starting [{__version__}]")
# # main(production_flag=True)
# main()
# print("Complete!")



In [ ]:
"""
Fetching data module. Aim is import all necessary fields up front, but if needed, you can define another function to be called here.

Usage:
    import src.cdutils.database

You need to set your own date that you want to see in effective date embedded in the SQL Query
"""
import cdutils.deduplication # type: ignore
import cdutils.database.connect # type: ignore
from sqlalchemy import text # type: ignore
import pandas as pd

def fetch_from_allroles():
    """
    Gets branch, primary officer and secondary officer
    """
    wh_allroles = text(f"""
    SELECT
        *
    FROM 
        OSIBANK.WH_ALLROLES a
    """)

    wh_pers = text(f"""
    SELECT
        *
    FROM
        OSIBANK.WH_PERS a
    """)

    queries = [
        {'key':'wh_allroles', 'sql':wh_allroles, 'engine':1},
        {'key':'wh_pers', 'sql':wh_pers, 'engine':1},
    ]


    data = cdutils.database.connect.retrieve_data(queries)
    return data

data = fetch_from_allroles()

wh_allroles = data['wh_allroles'].copy()
wh_pers = data['wh_pers'].copy()

selo = wh_allroles[wh_allroles['acctrolecd'] == 'SELO'].copy()

selo = selo.sort_values(by='datelastmaint', ascending=False).copy()
selo = cdutils.deduplication.dedupe([{'df':selo, 'field':'acctnbr'}])


pers = wh_pers.sort_values(by='datelastmaint', ascending=False).copy()
pers = cdutils.deduplication.dedupe([{'df':pers, 'field':'persnbr'}])

# Asserts
assert selo['acctnbr'].is_unique, "Failure"
assert pers['persnbr'].is_unique, "Failure"


selo = selo[['acctnbr','persnbr']].copy()
pers = pers[['persnbr','perssortname']].copy()


merged_df = pd.merge(selo, pers, on='persnbr',how='left')

merged_df = merged_df[['acctnbr','perssortname']].copy()
merged_df = merged_df.rename(columns={'acctnbr','perssortname'}).copy()




In [ ]:
"""
Main Entry Point
"""
from pathlib import Path

import pandas as pd # type: ignore

import src.fetch_data
import src.core_transform
import src.output_to_excel
import cdutils.pkey_sqlite # type: ignore
import cdutils.hhnbr # type: ignore
import cdutils.selo # type: ignore
from src._version import __version__

# def main(production_flag: bool=False):
#     if production_flag:
#         BASE_PATH = Path(r'\\00-DA1\Home\Share\Line of Business_Shared Services')
#         assert "prod" in __version__, (f"Cannot run in production mode without 'prod' in the __version__")
#     else:
#         BASE_PATH = Path('.')
    
data = src.fetch_data.fetch_data()

# Core transformation pipeline
raw_data = src.core_transform.main_pipeline(data)

# Raw data with pkey appended
raw_data = cdutils.pkey_sqlite.add_pkey(raw_data)

# Attach HH number
househldacct = data['househldacct'].copy()
raw_data = cdutils.hhnbr.add_hh_nbr(raw_data, househldacct)

# Attach secondary officer
df = cdutils.selo.append_selo(raw_data)

# # Output to excel (raw data)
BASE_PATH = Path('.')
OUTPUT_PATH = BASE_PATH / Path('./output/loan_report_branch_officer.xlsx')
df.to_excel(OUTPUT_PATH, sheet_name='Sheet1', index=False)

# # Format excel
# src.output_to_excel.format_excel_file(OUTPUT_PATH)

# if __name__ == '__main__':
# print(f"Starting [{__version__}]")
# # main(production_flag=True)
# main()
# print("Complete!")